In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sinr.graph_embeddings as ge
import sinr.text.evaluate as ev
#Subsample of OANC corpus as a co-occurrence matrix
sinr = ge.SINr.load_from_cooc_pkl("oanc_extracted_min_freq20_max_freq5900_prct_sampling1_matrix.pkl")

/lium/buster1/dugue/miniconda3-2023/envs/sinr-dev/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-19 17:48:01,117 - load_from_cooc_pkl - INFO - Building Graph.
2024-07-19 17:48:01,118 - load_pkl_text - INFO - Loading cooccurrence matrix and dictionary.
2024-07-19 17:48:01,152 - load_pkl_text - INFO - Finished loading data.
2024-07-19 17:48:03,631 - load_from_cooc_pkl - INFO - Finished building graph.


In [3]:
len(sinr.wrd_to_idx), "words of vocabulary in this subsample"

(17706, 'words of vocabulary in this subsample')

In [4]:
sinr.detect_communities(gamma=80)

2024-07-19 17:48:03,755 - detect_communities - INFO - Detecting communities.


Gamma for louvain : 80


2024-07-19 17:48:04,714 - detect_communities - INFO - Finished detecting communities.


Communities detected in 0.90354 [s]
solution properties:
-------------------  ------------
# communities        4933
min community size      1
max community size     32
avg. community size     3.5893
modularity              0.0345125
-------------------  ------------


In [5]:
sinr.extract_embeddings()

2024-07-19 17:48:04,767 - extract_embeddings - INFO - Extracting embeddings.
2024-07-19 17:48:04,767 - extract_embeddings - INFO - Applying NFM.
2024-07-19 17:48:04,769 - get_nfm_embeddings - INFO - Starting NFM
2024-07-19 17:48:30,492 - extract_embeddings - INFO - NFM successfully applied.
2024-07-19 17:48:30,494 - extract_embeddings - INFO - Finished extracting embeddings.


In [6]:
modele = ge.InterpretableWordsModelBuilder(sinr, "modele", n_jobs=8, n_neighbors=4).build()
ev.similarity_MEN_WS353_SCWS(modele)

94 missing words


27 missing words


235 missing words


{'MEN': 0.46094523263674375,
 'WS353': 0.4636897182566408,
 'SCWS': 0.42946339109424253}

In [7]:
modele.vectors.shape, "(nb of words, dimensions)"

((17706, 4933), '(nb of words, dimensions)')

In [8]:
#Small Subsample of OANC corpus as a co-occurrence matrix
sinr_small = ge.SINr.load_from_cooc_pkl("oanc_extracted_min_freq50_max_freq3000_prct_sampling08_matrix.pkl")

2024-07-19 17:48:39,768 - load_from_cooc_pkl - INFO - Building Graph.
2024-07-19 17:48:39,768 - load_pkl_text - INFO - Loading cooccurrence matrix and dictionary.
2024-07-19 17:48:39,787 - load_pkl_text - INFO - Finished loading data.
2024-07-19 17:48:41,251 - load_from_cooc_pkl - INFO - Finished building graph.


In [9]:
len(sinr_small.wrd_to_idx), "words of vocabulary"

(9494, 'words of vocabulary')

In [10]:
len(sinr.wrd_to_idx) - len(sinr_small.wrd_to_idx), "words are present in the big subsample, but absent in the small one"

(8212, 'words are present in the big subsample, but absent in the small one')

In [11]:
#Transferring the communities of the bigger model to the small one
sinr_small.transfert_communities_labels(modele.get_communities_as_labels_sets())

In [12]:
sinr_small.extract_embeddings()

2024-07-19 17:48:41,443 - extract_embeddings - INFO - Extracting embeddings.
2024-07-19 17:48:41,443 - extract_embeddings - INFO - Applying NFM.
2024-07-19 17:48:41,445 - get_nfm_embeddings - INFO - Starting NFM
2024-07-19 17:48:56,453 - extract_embeddings - INFO - NFM successfully applied.
2024-07-19 17:48:56,454 - extract_embeddings - INFO - Finished extracting embeddings.


In [13]:
modele_small = ge.InterpretableWordsModelBuilder(sinr_small, "modele", n_jobs=8, n_neighbors=4).build()
ev.similarity_MEN_WS353_SCWS(modele_small)

205 missing words


86 missing words


482 missing words


{'MEN': 0.3092365066427464,
 'WS353': 0.37938716672217637,
 'SCWS': 0.3762601899389452}

In [14]:
"Small model : ", modele_small.vectors.shape, "(nb of words, dimensions)"

('Small model : ', (9494, 4700), '(nb of words, dimensions)')

In [15]:
"Big modele : ", modele.vectors.shape, "(nb of words, dimensions)"

('Big modele : ', (17706, 4933), '(nb of words, dimensions)')

### Interpreting the small model in the big one's referential

The small model and its projected version into the big one's referential have the same neighborings, but their neighboring may be different from the big one.

In [16]:
modele_small_bigasaref = modele.get_vectors_using_self_space(modele_small)

In [17]:
modele_small_bigasaref.most_similar("cheese")

{'object ': 'cheese',
 'neighbors ': [('bread', 0.37), ('wine', 0.35), ('bottle', 0.32)]}

In [18]:
modele_small.most_similar("cheese")

{'object ': 'cheese',
 'neighbors ': [('bread', 0.37), ('wine', 0.35), ('bottle', 0.32)]}

In [19]:
modele.most_similar("cheese")

{'object ': 'cheese',
 'neighbors ': [('foodstuff', 0.49), ('sausage', 0.45), ('bread', 0.41)]}

The small model and its projected version into the big one's referential have the same stereotypes, but the dimensions ids are different. The dimensions ids of the projected versions are aligned with the big model.

In [20]:
modele.get_obj_stereotypes("hippocampus")

[{'dimension': 769,
  'value': True,
  'stereotypes': [(0.23, 'hippocampus'),
   (0.22, 'amygdala'),
   (0.21, 'furrow')]},
 {'dimension': 1968,
  'value': True,
  'stereotypes': [(0.38, 'tclr'), (0.1, 'spleen'), (0.09, 'intestine')]},
 {'dimension': 1944,
  'value': True,
  'stereotypes': [(0.34, 'cord'), (0.32, 'l6-s1'), (0.3, 'spinal')]},
 {'dimension': 1196,
  'value': True,
  'stereotypes': [(0.1, 'autistic'), (0.06, 'amyloid'), (0.06, 'amygdala')]},
 {'dimension': 291,
  'value': True,
  'stereotypes': [(0.47, 'rdta'), (0.36, 'scid'), (0.3, 'balb')]}]

In [21]:
modele_small.get_obj_stereotypes("hippocampus")

[{'dimension': 1812,
  'value': True,
  'stereotypes': [(0.26, 'furrow'), (0.23, 'hippocampus'), (0.2, 'cortex')]},
 {'dimension': 757,
  'value': True,
  'stereotypes': [(0.11, 'cirrhosis'), (0.11, 'intestine'), (0.09, 'spleen')]},
 {'dimension': 1494,
  'value': True,
  'stereotypes': [(0.35, 'cord'), (0.29, 'spinal'), (0.28, 'dorsal')]},
 {'dimension': 1673,
  'value': True,
  'stereotypes': [(0.03, 'hippocampus'), (0.03, 'lifelong'), (0.03, 'diet')]},
 {'dimension': 2862,
  'value': True,
  'stereotypes': [(0.12, 'translocation'),
   (0.07, 'localization'),
   (0.07, 'chloroplast')]}]

In [22]:
modele_small_bigasaref.get_obj_stereotypes("hippocampus")

[{'dimension': 769,
  'value': True,
  'stereotypes': [(0.26, 'furrow'), (0.23, 'hippocampus'), (0.2, 'cortex')]},
 {'dimension': 1968,
  'value': True,
  'stereotypes': [(0.11, 'cirrhosis'), (0.11, 'intestine'), (0.09, 'spleen')]},
 {'dimension': 3455,
  'value': True,
  'stereotypes': [(0.03, 'hippocampus'), (0.03, 'lifelong'), (0.03, 'diet')]},
 {'dimension': 1944,
  'value': True,
  'stereotypes': [(0.35, 'cord'), (0.29, 'spinal'), (0.28, 'dorsal')]},
 {'dimension': 1670,
  'value': True,
  'stereotypes': [(0.12, 'phytoestrogen'), (0.1, 'rodent'), (0.1, 'pufa')]}]

In [23]:
modele.get_obj_stereotypes("beverage")

[{'dimension': 640,
  'value': True,
  'stereotypes': [(0.12, 'caloric'), (0.11, 'beverage'), (0.08, 'intake')]},
 {'dimension': 641,
  'value': True,
  'stereotypes': [(0.1, 'beverage'), (0.08, 'alcoholic'), (0.08, 'foc')]},
 {'dimension': 4784,
  'value': True,
  'stereotypes': [(0.18, 'overdominance'),
   (0.16, 'k1'),
   (0.11, 'phosphatidylinositol')]},
 {'dimension': 713,
  'value': True,
  'stereotypes': [(0.1, 'bottled'), (0.05, 'gin'), (0.04, 'vodka')]},
 {'dimension': 439,
  'value': True,
  'stereotypes': [(0.17, 'parenchyma'),
   (0.15, 'extravascular'),
   (0.13, 'artifactual')]}]

In [24]:
modele_small.get_obj_stereotypes("beverage")

[{'dimension': 2220,
  'value': True,
  'stereotypes': [(0.14, 'beverage'), (0.09, 'intake'), (0.06, 'calorie')]},
 {'dimension': 183,
  'value': True,
  'stereotypes': [(0.11, 'beverage'), (0.07, 'alcoholic'), (0.02, 'citrus')]},
 {'dimension': 1362,
  'value': True,
  'stereotypes': [(0.04, 'alcoholic'), (0.03, 'beverage'), (0.02, 'liquor')]},
 {'dimension': 524,
  'value': True,
  'stereotypes': [(0.05, 'antioxidant'), (0.04, 'fortify'), (0.04, 'bottle')]},
 {'dimension': 2511,
  'value': True,
  'stereotypes': [(0.07, 'adherent'),
   (0.07, 'emphysema'),
   (0.06, 'neutrophil')]}]

In [25]:
modele_small_bigasaref.get_obj_stereotypes("beverage")

[{'dimension': 640,
  'value': True,
  'stereotypes': [(0.14, 'beverage'), (0.09, 'intake'), (0.06, 'calorie')]},
 {'dimension': 641,
  'value': True,
  'stereotypes': [(0.11, 'beverage'), (0.07, 'alcoholic'), (0.02, 'citrus')]},
 {'dimension': 713,
  'value': True,
  'stereotypes': [(0.04, 'alcoholic'), (0.03, 'beverage'), (0.02, 'liquor')]},
 {'dimension': 1618,
  'value': True,
  'stereotypes': [(0.05, 'antioxidant'), (0.04, 'fortify'), (0.04, 'bottle')]},
 {'dimension': 439,
  'value': True,
  'stereotypes': [(0.07, 'adherent'),
   (0.07, 'emphysema'),
   (0.06, 'neutrophil')]}]